# 语言结构种的递归

## 用级联分块器构建嵌套结构

到目前为止，我们的块结构一直都是相对平的，已标注标识符组成的树在如 NP 这样的块节点下任意组合。然而，只需创建一个包含递归规则的多级块语法，就可以建立任意深度的块结构。下例是名词短语、介词短语、动词短语和句子的模式，这是一个四级块语法器，可以用来创建深度最多为 4 的结构：

In [1]:
import nltk

grammar = r"""
    NP: {<DT|JJ|NN.*>+}
    PP: {<IN><NP>}
    VP: {<VB.*><NP|PP|CLAUSE>+$}
    CLAUSE: {<NP><VP>}
"""

cp = nltk.RegexpParser(grammar)
sentence = [("Mary", "NN"), ("saw", "VBD"), ("the", "DT"), ("cat", "NN"),
    ("sit", "VB"), ("on", "IN"), ("the", "DT"), ("mat", "NN")]
print(cp.parse(sentence))

(S
  (NP Mary/NN)
  saw/VBD
  (CLAUSE
    (NP the/DT cat/NN)
    (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))


不幸的是，这一结果丢掉了 saw 开头的 VP 结构。当我们将此分块器应用到一个有更深嵌套的句子时，可以看到多层结构都无法识别。

In [2]:
sentence = [("John", "NNP"), ("thinks", "VBZ"), ("Mary", "NN"),
            ("saw", "VBD"), ("the", "DT"), ("cat", "NN"), ("sit", "VB"),
            ("on", "IN"), ("the", "DT"), ("mat", "NN")]
print(cp.parse(sentence))

(S
  (NP John/NNP)
  thinks/VBZ
  (NP Mary/NN)
  saw/VBD
  (CLAUSE
    (NP the/DT cat/NN)
    (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))


这些问题的解决方案是：让分块器在它的模式中循环，即尝试完所有模式之后，重复此过程。我们添加一个可选的参数 loop 来指定这套模式应该循环的次数。

In [3]:
cp = nltk.RegexpParser(grammar, loop=2)
print(cp.parse(sentence))

(S
  (NP John/NNP)
  thinks/VBZ
  (CLAUSE
    (NP Mary/NN)
    (VP
      saw/VBD
      (CLAUSE
        (NP the/DT cat/NN)
        (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))))


## 树

树是一组连接的加标签节点，从一个特殊的根节点沿一条唯一的路径到达每个节点。下面是一棵树的例子，其中 S 是 VP 的父节点，VP 是 S 的一个子节点，上层的 NP 和 VP 互为兄弟节点：

![ch07-tree-3.png](resources/ch07-tree-3.png)

在 NLTK 中，我们通过给一个节点添加标签和孩子链表的方式创建一颗树 [nltk.Tree](https://www.nltk.org/_modules/nltk/tree.html)：

In [4]:
tree1 = nltk.Tree('NP', ['Alice'])
print(tree1)
tree2 = nltk.Tree('NP', ['the', 'rabbit'])
print(tree2)

(NP Alice)
(NP the rabbit)


我们可以将这些不断合并成为更大的树：

In [5]:
tree3 = nltk.Tree('VP', ['chased', tree2])
tree4 = nltk.Tree('S', [tree1, tree3])
print(tree4)

(S (NP Alice) (VP chased (NP the rabbit)))


下面是访问树对象的一些方法，其中 label 方法返回当前节点的标签，leaves 方法返回当前节点的所有叶节点，索引 i 返回树的第 i 个子节点：

In [6]:
print(tree4[1])
print(tree4[1].label())
print(tree4.leaves())
print(tree4[1][1][1])

(VP chased (NP the rabbit))
VP
['Alice', 'chased', 'the', 'rabbit']
rabbit


复杂的树用括号难以阅读，可以使用 draw 方法进行图形化展示：

In [7]:
tree3.draw()

## 树遍历

标准做法是使用递归函数来遍历树。注：[Tree.fromstring](https://www.nltk.org/_modules/nltk/tree.html#Tree.fromstring) 方法可以直接通过括号表示的树字符串构造树结构。

In [8]:
def traverse(t):
    try:
        t.label()
    except AttributeError:
        print(t, end=' ')
    else:
        # Now we know that t.node is defined
        print('(', t.label(), end=' ')
        for child in t:
            traverse(child)
        print(')', end=' ')

t = nltk.Tree.fromstring('(S (NP Alice) (VP chased (NP the rabbit)))')
traverse(t)

( S ( NP Alice ) ( VP chased ( NP the rabbit ) ) ) 